# MAD Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from scipy.optimize import least_squares

from scipy import ndimage
import os
import scipy.io as sio
from scipy.ndimage import rotate
from skimage.io import imread


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.linalg import lstsq


In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x000001E07B4CA390>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [3]:
# SENSIBILITE CAPTEUR
S_force =  39.2  # mV/N
G_force = 100

# cannal d'acquisition
Cannal = 1

In [4]:
dossier = 'mes_2024.03.22'
nom_anche = '/A'
nbr_anche = 3
seuil = 150

## Traitement données

In [5]:
for i in range(nbr_anche):
    # Metre le fichier . mat en .txt        
    # Lecture du fichier MATLAB (.m) et écriture dans un fichier texte (.txt)
    with open(dossier + f'/A{i+1}.mat', 'r') as file:
        matlab_code = file.read()
        
    # Écriture du contenu dans un fichier texte
    with open(dossier + f'/A{i+1}.txt', 'w') as file:
        file.write(matlab_code)
        
    # Ouvrir le fichier d'entrée en mode lecture
    with open(dossier + f'/A{i+1}.txt', 'r') as f:
        lines = f.readlines()

    # Initialiser une variable pour suivre le numéro de fichier en cours
    current_file_index = 1

    # Parcourir les lignes du fichier
    for line in lines:
        if line.startswith('#'):
            # Si une ligne commence par '#', cela indique un nouveau fichier
            current_file_index += 1
        else:
            # Écrire le contenu de la ligne dans le fichier correspondant
            with open(dossier + f'/matrix{current_file_index}_A{i+1}.txt', 'a') as current_file:
                current_file.write(line)
    
    # def_r 
    with open(dossier + f'/matrix6_A{i+1}.txt', 'r') as f:
        lines = f.readlines()

    # Initialiser une variable pour suivre le numéro de fichier en cours
    current_file_index = 1
    # Parcourir les lignes du fichier
    for line in lines:
        # Diviser la ligne en valeurs individuelles en utilisant l'espace comme délimiteur
        values = line.split()
        values = np.array(values)
        # Écrire les valeurs dans le fichier correspondant
        with open(dossier + f'/def_r_A{i+1}.txt', 'a') as current_file:
            for value in values:
                current_file.write(value + ' ')
            current_file.write('\n')

    # der_m
    with open(dossier + f'/matrix10_A{i+1}.txt', 'r') as f:
        lines = f.readlines()
    current_file_index = 1
    for line in lines:
        values = line.split()
        values = np.array(values)
        with open(dossier + f'/def_m_A{i+1}.txt', 'a') as current_file:
            for value in values:
                current_file.write(value + ' ')
            current_file.write('\n')
            
    # surf_c
    with open(dossier + f'/matrix14_A{i+1}.txt', 'r') as f:
        lines = f.readlines()
    current_file_index = 1
    for line in lines:
        values = line.split()
        values = np.array(values)
        with open(dossier + f'/surf_c_A{i+1}.txt', 'a') as current_file:
            for value in values:
                current_file.write(value + ' ')
            current_file.write('\n')

In [6]:
# Metre le fichier . mat en .txt        
# Lecture du fichier MATLAB (.m) et écriture dans un fichier texte (.txt)
with open(dossier + '/A1.mat', 'r') as file:
    matlab_code = file.read()
    
# Écriture du contenu dans un fichier texte
with open(dossier + '/A1.txt', 'w') as file:
    file.write(matlab_code)


In [8]:
# Ouvrir le fichier d'entrée en mode lecture
with open(dossier + '/A1.txt', 'r') as f:
    lines = f.readlines()

# Initialiser une variable pour suivre le numéro de fichier en cours
current_file_index = 1

# Parcourir les lignes du fichier
for line in lines:
    if line.startswith('#'):
        # Si une ligne commence par '#', cela indique un nouveau fichier
        current_file_index += 1
    else:
        # Écrire le contenu de la ligne dans le fichier correspondant
        with open(dossier + f'/matrix{current_file_index}.txt', 'a') as current_file:
            current_file.write(line)

In [9]:
# def_r 
with open(dossier + '/matrix6.txt', 'r') as f:
    lines = f.readlines()

# Initialiser une variable pour suivre le numéro de fichier en cours
current_file_index = 1
# Parcourir les lignes du fichier
for line in lines:
    # Diviser la ligne en valeurs individuelles en utilisant l'espace comme délimiteur
    values = line.split()
    values = np.array(values)
    # Écrire les valeurs dans le fichier correspondant
    with open(dossier + '/def_r.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')

# der_m
with open(dossier + '/matrix10.txt', 'r') as f:
    lines = f.readlines()
current_file_index = 1
for line in lines:
    values = line.split()
    values = np.array(values)
    with open(dossier + '/def_m.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')
        
# surf_c
with open(dossier + '/matrix14.txt', 'r') as f:
    lines = f.readlines()
current_file_index = 1
for line in lines:
    values = line.split()
    values = np.array(values)
    with open(dossier + '/surf_c.txt', 'a') as current_file:
        for value in values:
            current_file.write(value + ' ')
        current_file.write('\n')

# Traitement images

### Extraction données image

In [10]:
# FONCTION DE ROTATION D'IMAGE 
# entrée : une image
# ginput : bord gauche et bord droit du bec
# sortie : angle de rotation, image J après rotation, coordonées des points selectionnés
def im_rotate(image):
    plt.imshow(image, cmap='gray')
    plt.title("Cliquez sur les bords du bec (gauche puis droit)")

    # récuperer les coordonnées des bords du bec
    bec_coord = plt.ginput(2)
    plt.close()
    # print(f"Cordonnées des bords du bec: {np.round(bec_coord, 2)}")
        
    # Calculer l'angle pour aligner l'image
    (x1, y1), (x2, y2) = bec_coord
    rot_angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    # print(f"Angle à corriger: {rot_angle} degrés")

    # Calculer le centre de l'image
    (h, w) = image.shape
    center = (w // 2, h // 2)

    # Créer une matrice de rotation centrale
    rot_mat = cv2.getRotationMatrix2D(center, rot_angle, 1.0)

    # Appliquer la rotation
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)

    # plt.imshow(img_rot, cmap='gray')
    # plt.title("Image alignée")
    
    return rot_mat, img_rot, bec_coord

# ROGNER IMAGE
# entrée : image, coordonées du bec (_,_, xy_bec = im_rotate(J))
def rogner(image, xy_bec):
    # img_rogn = np.flipud(image)
    x_BG = xy_bec[0][0]
    y_BG = xy_bec[0][1] 
    x_BD = xy_bec[1][0]
    y_BD = xy_bec[1][1]
    
        # bornes 
    wmin = int(x_BG)-20     # width min
    wmax = int(x_BD)+20
    hmin = int(y_BG-60)     # height min
    hmax = int(y_BG+100)
    img_rogn = image[hmin:hmax, wmin:wmax] 
    
    return img_rogn

def bord_anche(image):
    points = []
    I = np.flipud(image)
    (H, W) = I.shape
    # print(H,W)
    plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    plt.axis('equal')
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.title("Cliquez sur les bords de l'anche")
    anche_coord = plt.ginput(2)
    plt.close()
    return anche_coord

# CALCUL DU CANAL D'nbr_anche
# entrée :  image, coordonnées des bords de l'anche,
def surface_canal(image, seuil, anche_coord):
    # s'assurer que I est bien en binaire
    _, I = cv2.threshold(image, seuil, 255, cv2.THRESH_BINARY)

    # # coordonnées des bords du bec    
    # x_BG = xy_bec[0][0]
    # y_BG = xy_bec[0][1] 
    # x_BD = xy_bec[1][0]
    # y_BD = xy_bec[1][1]

    # # bornes de l'image
    # wmin = int(x_BG)-20     # width min
    # wmax = int(x_BD)+20
    # hmin = int(y_BG-60)     # height min
    # hmax = int(y_BG+100)
    
    (x1, y1), (x2, y2) = map(lambda p: (int(p[0]), int(p[1])), anche_coord)
    # initialisation matrices vides
    Xv = np.arange(x1, x2, dtype=int)   # vecteur de la taille de la largeur de l'image coupée
    Yv_m = []
    Yv_r = []
    

# PROBLEME POUR GENERALISER A UNE TRANCHE ENTIERE
    for xv in Xv:
        Iv = I[:, xv].astype(float)   # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
        Iv_2 = np.round(np.gradient(Iv)).astype(int)     # Dérivée centrée (flottante) + Arrondir + convertir en entiers
       
        # seuillage trance par tranche
        smin = -30
        smax = -smin
        pos1 = np.where(Iv_2 < smin)[0][0] if any(Iv_2 < smin) else None
        pos2 = np.where(Iv_2 > smax)[0][-1] if any(Iv_2 > smax) else None
        (H, W) = I.shape
        Y = np.arange(1,H+1)
        dist_px = 5
    
        if pos2 is not None:
            # Yv_m[i] = Y[pos2]
            Yv_m.append(Y[pos2])
        else:
            # Yv_m[i] = np.nan
            Yv_m.append(np.nan)
            # print('error!!!')  # Peu probable parce que détectera toujours la frontière lèvre/anche

        if pos1 is not None:
            # Yv_r[i] = Y[pos1]
            Yv_r.append(Y[pos1])
        else:
            # Yv_r[i] = Yv_m[i]  # Lorsque le canal d'anche est fermé, il n'y aura pas de Yv_r. On lui donne la même valeur que Yv_m comme ça la différence sera de 0.
            Yv_r.append(Yv_m)
            
    # Calculer la différence entre les deux courbes
    diff = [m - r for m, r in zip(Yv_m, Yv_r)]
    diff = np.abs(diff)
    # les Nan sont remplacés par des 0
    diff = np.nan_to_num(diff)

    # Calculer l'aire en utilisant la méthode des trapèzes en pixel²
    area = np.trapz(diff, Xv)
    
    # paramètre caméra
    dist_px = 1064  # mesuré sur l'image MIRE 05-03-42 (8 carreaux)
    dist_mm = 12  # mesuré à la règle sur la mire (cf mon cahier de labo, 2 carreaux = 3mm)
    S = dist_mm / dist_px  # "sensibilité" (en mm/px)

    # mise en mm²
    surf_pix = S ** 2  # surface d'un pixel
    dim_canal = area * surf_pix  # surface en mm² du canal
        
    return Xv, Yv_r, Yv_m, dim_canal



In [11]:
J = cv2.imread(dossier + nom_anche + '1_0.bmp', cv2.IMREAD_GRAYSCALE)
_, J = cv2.threshold(J, seuil, 255, cv2.THRESH_BINARY)
J = J.astype(np.float32)
plt.imshow(J, cmap='gray')

In [13]:
plt.close('all')


# Initialisation sur une image

# Lecture de l'image en nuance de gris, mise en noir et blanc
J = cv2.imread(dossier + nom_anche + '1_0.bmp', cv2.IMREAD_GRAYSCALE)
_, J = cv2.threshold(J, seuil, 255, cv2.THRESH_BINARY)
(h, w) = J.shape

rot_mat, J_rot,  xy_bec = im_rotate(J)
I = rogner(J_rot, xy_bec)
anche_coord = bord_anche(I)

nbr_force = [29, 18, 46]

# Import data
for i in range(nbr_anche):
    with open(dossier + f'/air_canal_A{i+1}.txt', 'w') as file:
        dim_c = []
        for f in range(50):
            img_path = dossier + nom_anche + f'{i+1}_{f}.bmp'
            if not os.path.exists(img_path):
                # print(f"File {img_path} does not exist. Skipping to next iteration.")
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)
            img_rot = cv2.warpAffine(img, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            img =  rogner(img, xy_bec)
            # img = np.flipud(img)
            Xv, Yv_r, Yv_m, dim_canal = surface_canal(img, seuil, anche_coord)
            dim_c.append(dim_canal)
            plot_fig = Flase
            if plot_fig :
                plt.figure()
                plt.imshow(img, cmap='gray')
                plt.plot(Xv, Yv_r, '.-', linewidth=0.1)
                plt.plot(Xv, Yv_m, 'm.-', linewidth=0.1)
                print(f"L'aire entre les deux courbes est : {np.round(dim_canal, 2)} mm²")

            file.write(str(dim_canal) + '\n')
    

L'aire entre les deux courbes est : 12.25 mm²
L'aire entre les deux courbes est : 12.13 mm²
L'aire entre les deux courbes est : 11.81 mm²
L'aire entre les deux courbes est : 11.39 mm²
L'aire entre les deux courbes est : 10.9 mm²
L'aire entre les deux courbes est : 10.37 mm²
L'aire entre les deux courbes est : 9.83 mm²
L'aire entre les deux courbes est : 9.27 mm²
L'aire entre les deux courbes est : 8.68 mm²
L'aire entre les deux courbes est : 8.11 mm²
L'aire entre les deux courbes est : 7.51 mm²
L'aire entre les deux courbes est : 6.97 mm²
L'aire entre les deux courbes est : 6.42 mm²
L'aire entre les deux courbes est : 5.89 mm²
L'aire entre les deux courbes est : 5.38 mm²
L'aire entre les deux courbes est : 4.83 mm²
L'aire entre les deux courbes est : 4.27 mm²
L'aire entre les deux courbes est : 3.76 mm²
L'aire entre les deux courbes est : 3.26 mm²
L'aire entre les deux courbes est : 2.74 mm²


C:\Users\Admin\AppData\Local\Temp\ipykernel_2136\555632624.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


L'aire entre les deux courbes est : 2.25 mm²
L'aire entre les deux courbes est : 1.73 mm²
L'aire entre les deux courbes est : 1.31 mm²
L'aire entre les deux courbes est : 1.02 mm²
L'aire entre les deux courbes est : 0.74 mm²
L'aire entre les deux courbes est : 0.5 mm²
L'aire entre les deux courbes est : 0.27 mm²
L'aire entre les deux courbes est : 0.02 mm²
L'aire entre les deux courbes est : 0.0 mm²
L'aire entre les deux courbes est : 0.0 mm²
L'aire entre les deux courbes est : 12.7 mm²
L'aire entre les deux courbes est : 12.61 mm²
L'aire entre les deux courbes est : 12.05 mm²
L'aire entre les deux courbes est : 11.23 mm²
L'aire entre les deux courbes est : 10.33 mm²
L'aire entre les deux courbes est : 9.4 mm²
L'aire entre les deux courbes est : 8.42 mm²
L'aire entre les deux courbes est : 7.44 mm²
L'aire entre les deux courbes est : 6.42 mm²
L'aire entre les deux courbes est : 5.38 mm²
L'aire entre les deux courbes est : 4.41 mm²
L'aire entre les deux courbes est : 3.5 mm²
L'aire entr

In [16]:
plt.close('all')
# plt.figure(figsize=(10,6))
plt.figure()

for i in range(nbr_anche):
    force = np.loadtxt(dossier + f'/force_A{i+1}.txt')
    surface_canal = np.loadtxt(dossier + f'/air_canal_A{i+1}.txt')
    plt.plot(force, surface_canal, '*-', linewidth=1.8, label='Force 1')
    # identification_modele(F, surf_c)
    plt.xlabel('Force')
    plt.ylabel('Surface (mm²)')
    plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
    plt.legend()
    plt.grid(True)
    plt.axis([-30, 350, -2, 13])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

### Modèle

In [10]:
# Caracteristique non lineaire S(p)
# p : Pression dans la bouche
# S00 : Ouverture a p=0
# pM : Pression de plaquage
# Coude : Etendue de la partie coude en Pa
# Fuite : Ouverture equivalente a la fuite


def caracNL_BG(p, param):
    S00 = param[0]
    pM = param[1]
    Pb = param[2]
    SL = param[3]

    # Calcul de la pente de la droite linéaire
    C = S00 / pM
    PprimeM = (S00 - SL) / C
    A = (S00 - SL) / (4 * Pb * PprimeM)

    pb1 = PprimeM - Pb  # pression début coude
    pb2 = PprimeM + Pb  # pression fin coude

    # Création des indices pour différentes parties de la courbe
    ind1 = np.where(p <= pb1)  # partie linéaire
    ind2 = np.where((p > pb1) & (p < pb2))  # coude
    ind3 = np.where(p >= pb2)  # fuite

    # Initialisation de la section S
    S = np.zeros_like(p)

    # Calcul des valeurs de S pour les différentes parties de la courbe
    S[ind1] = S00 - C * p[ind1]  # partie linéaire avant enroulement
    S[ind2] = A * (p[ind2] - PprimeM - Pb) ** 2 + SL  # coude -> équation du second degré
    S[ind3] = SL  # fuite
    print('S = ', S)
    return S


In [11]:
# pg : pression généralisée (= force de la lèvre )
# S : section d'ouverture
# S00 : section d'ouverture à pg nulle (ordonnée à l'origine)
# PM: pression de plaquage
# Coude : Pb dans le modèle
# Sfuites : section de fuites
# Paramopt : parametres optimaux [S00, PM, Coude, Sfuites]
# Smodelepg: section théorique correspondant aux parametres optimaux

def identification_modele(pg, S):
    # Identification grossière des paramètres pour l'initialisation de l'optimisation
    Smax = max(S)
    indmax = np.where(S > 0.5 * Smax)
    
    # Ajustement linéaire pour estimer S00 et Cest
    P = np.polyfit(pg[indmax], S[indmax], 1)
    Cest = -P[0]  # pente
    S0est = P[1]  # ordonnée à l'origine
    Stheo = np.polyval(P, pg[indmax])

    # Paramètres initiaux de l'optimisation
    PM0 = S0est / Cest
    Sfuites0 = 0
    Coude0 = PM0 / 4
    p0 = [S0est, PM0, Coude0, Sfuites0]

    # Minimisation au sens des moindres carrés
    Paramopt, _ = curve_fit(caracNL_BG, pg, S, p0)
    Smodelepg = caracNL_BG(pg, *Paramopt)
    erreuropt = np.sqrt(np.sum((Smodelepg - S) ** 2) / np.sum(S ** 2)) * 100

    # Tracé des résultats
    pgt = np.linspace(0, 6, 100)
    ST = caracNL_BG(pgt, *Paramopt)
    
    plt.plot(pg, S, '-+', linewidth=2, label='Données expérimentales')
    plt.plot(pgt, ST, 'k', linewidth=3, label='Modèle ajusté')
    plt.plot(0, Paramopt[0], '+r', LineWidth=6)
    plt.plot(Paramopt[1], 0, '+r', LineWidth=6)
    plt.plot([Paramopt[1], 0], [0, Paramopt[0]], '-.r', LineWidth=1)
    plt.plot(Paramopt[1] + Paramopt[2], 0, '+g', LineWidth=6)
    plt.plot(Paramopt[1] - Paramopt[2], 0, '+g', LineWidth=6)
    plt.plot(0, Paramopt[3], '+b', LineWidth=6)
    plt.plot([0, 6], [Paramopt[3], Paramopt[3]], ':b', LineWidth=1)
    
    limit = np.linspace(0, 14, 100)
    plt.plot((Paramopt[1] + Paramopt[2]) * np.ones_like(limit), limit, '--g', linewidth=1)
    plt.plot((Paramopt[1] - Paramopt[2]) * np.ones_like(limit), limit, '--g', linewidth=1)
    
    plt.xlabel('Pression généralisée (pg)')
    plt.ylabel('Section d\'ouverture (S)')
    plt.legend()
    plt.show()

    return Paramopt, erreuropt


In [12]:
force = np.loadtxt(dossier + '/force_A1.txt')
surface_canal = np.loadtxt(dossier + '/air_canal_A1.txt')
Paramopt, erreuropt = identification_modele(force, surface_canal)
print(f'Paramètres optimaux : {Paramopt}')
print(f'Erreur optimale : {erreuropt:.2f}%')


TypeError: caracNL_BG() takes 2 positional arguments but 5 were given

### Traitement des données extraites

In [11]:
# tracer la surface du cannal d'anche

plt.close('all')

plt.figure(figsize=(8, 6))
dossier = 'MAD Statique/mes_2024.03.22'
pentes = []
# surf_c = []
# F =[]
for i in range (3): 
    surf_c = np.loadtxt(dossier + f'/A{i+1}.txt')
    F = np.loadtxt(dossier + f'/force_A{i+1}.txt')
    
 
    # PlotF
    plt.plot(F, surf_c, '*-', linewidth=1.8, label=f'Force {i+1}')
    # identification_modele(F, surf_c)
    plt.xlabel('Force')
    plt.ylabel('Surface (mm²)')
    plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
    plt.legend()
    plt.grid(True)
    plt.axis([-30, 350, -2, 13])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


FileNotFoundError: MAD Statique/mes_2024.03.22/A1.txt not found.